<center><h1>MCD: Ciencia de Datos Responsable</h1><center>
<h2>Tutorial 13 - Privacidad Diferencial</h2>


En este tutorial introduciremos el concepto de Privacidad Diferencial, técnica que tiene por objetivo proteger la privacidad de los datos mientras se preserva la utilidad de estos para ser analizados.

## Introducción

En sí, la privacidad diferencial constituye una promesa de los responsables de los datos:  "No serás afectado de manera adversa o de otro modo al permitir que tus datos sean utilizados en cualquier estudio."

Uno de los enfoques más simples para la privacidad diferencial es "Randomized Response" o respuesta aleatorizada o randomizada. Supongamos que necesitamos acceder a información sensible mediante una encuesta, por ejemplo, una encuesta sobre infidelidad. Es claro que muchas personas tenderán a negarse a responder esta encuesta si su privacidad no está garantizada. ¿Cómo podemos cuantificar qué fracción de la población es infiel (en este ejemplo), asegurando que la privacidad de los participantes estará a salvo?

Vamos caso por caso. Tengamos en consideración que la respuesta a esta pregunta solo toma valores sí/no:

- La primera opción sería recibir las respuestas verdaderas de cada encuestado; en este caso, la respuesta es igual a la verdad del encuestado y por ende no tenemos privacidad.

- La segunda opción es que se responda sí/no de manera aleatoria, ambos con probabilidad 0,5. En este caso la privacidad es "perfecta", pues la respuesta **no tiene relación con la verdad**, es decir, no podemos deducir la verdadera situación de un encuestado viendo su respuesta, pues independiente de su verdad la probabilidad de responder sí o no es la misma. Por ende la encuesta pierde también su accuracy, y no es posible realizar inferencias a partir de esta.

- Para la tercera opción, denotemos por $X_i$ (1: sí, 0: no) la verdad del encuestado, y por $Y_i$ (1: sí, 0: no) la respuesta de este. Pensemos en el siguiente esquema de respuesta:
<center>
$Y_i =
\begin{cases}
  X_i, & \text{con probabilidad } \frac{1}{2} + \gamma \\
  1 - X_i , & \text{con probabilidad } \frac{1}{2} - \gamma
\end{cases}$<center>

Es decir, el encuestado dice la verdad con probabilidad $\frac{1}{2} + \gamma$ y miente con probabilidad $\frac{1}{2} - \gamma$. Notemos que en este caso existe lo que llamamos "negabilidad plausible" pues si tomamos un valor de, por ejemplo $\gamma = 0.25$ , la respuesta del encuestado será la verdad con probabilidad 0.75 y no será la verdad con probabilidad 0.25. Qué tan negable es la respuesta otorgada está relacionado directamente con el nivel de privacidad.

## Aplicación de Randomized Response con lanzamiento de monedas.

Consideremos nuevamente la pregunta: ¿Es usted infiel? y considere el siguiente esquema de respuesta:

![Randomized Response Coin Flip](https://www.researchgate.net/publication/359729213/figure/fig1/AS:1141344529592321@1649129149405/Summary-of-randomized-response-method-with-unbiased-coins-ie-with-equal-1-2.png)

Supongamos entonces que una persona ha sido infiel. ¿Con qué probabilidad responderá que ha sido infiel? Tenemos que:
<center>
$P(Y_i = 1 | X_i = 1) = P(Cara)\cdot\frac{1}{2} + P(Sello)$

<center>
$\Leftrightarrow P(Y_i = 1 | X_i = 1) = \frac{1}{2}\cdot\frac{1}{2} + \frac{1}{2} = 0.75$</center>


De forma similar, podemos calcular la posibiilidad de que una persona diga que no ha sido infiel dado que sí lo ha sido:


<center>
$P(Y_i = 0 | X_i = 1) = P(Cara)\cdot\frac{1}{2} = 0.25$


Simulemos como sería el algoritmo de respuesta en este caso:

In [ ]:
import numpy as np

def CoinFlipRR(X):

  # Lanzamos la primera moneda
  coin_1 = np.random.choice(['Cara', 'Sello'], p=[0.5, 0.5])

  # Si la primera respuesta fue sello, se debe responder con la verdad.
  if coin_1 == 'Sello':
    y = X
    return y

  else:
    coin_2 = np.random.choice(['Cara', 'Sello'], p=[0.5, 0.5])

    # Supondremos que con cara se debe responder que sí, independiente de la verdad.
    if coin_2 == 'Cara':
      y = 1
      return y

    # Supondremos que con sello se debe responder que no, independiente de la verdad.
    else:
      y = 0
      return y

In [ ]:
import time

for i in range(10):
  print(f'Simulación de respuesta {i + 1}: {CoinFlipRR(1)}')
  time.sleep(0.5)

Simulación de respuesta 1: 1
Simulación de respuesta 2: 0
Simulación de respuesta 3: 1
Simulación de respuesta 4: 1
Simulación de respuesta 5: 0
Simulación de respuesta 6: 1
Simulación de respuesta 7: 1
Simulación de respuesta 8: 0
Simulación de respuesta 9: 1
Simulación de respuesta 10: 0


## Estimación de las cantidades verdaderas.

Ahora buscamos, a partir del número de respuestas recibidas, estimar las cantidades verdaderas (en este caso la proporción de personas infieles dentro de las encuestadas). Denotemos por $R_+$ la cantidad de encuestados que responden afirmativamente, $N_{-}$ la cantidad de encuestados que no ha sido infiel, $N_{+}$ la cantidad de encuestados que sí ha sido infiel y $n$ el total de encuestados. Luego tenemos que:

$R_+ = 0.25\cdot N_{-} + 0.75 \cdot N_{+}$


$\Leftrightarrow R_+ = 0.25\cdot(n-N_{+}) + 0.75\cdot N_{+}$

$\Leftrightarrow R_+ = 0.25\cdot n- 0.25 \cdot N_{+} + 0.75\cdot N_{+}$

$\Leftrightarrow R_+ = 0.25\cdot n + 0.5\cdot N_{+}$

$\Leftrightarrow 2\cdot R_+ = 0.5\cdot n + N_{+}$

$\Leftrightarrow N_{+} = 2 \cdot R_{+} - 0.5 \cdot n$

donde $N_{+}$ es la estimación de la cantidad de encuestados que ha sido infiel. Luego, haciendo uso del método de Randomized Response, podemos estimar la proporción de los encuestados que es infiel asegurando cierto nivel de privacidad. Finalmente, cabe preguntarse ¿Cómo cuantificamos el nivel de privacidad otorgado por este método? Para ello debemos dar una definición más formal de privacidad diferencial.

Diremos que un algoritmo $M$ (en este caso Randomized Response) es $\epsilon$-diferencialmente privado si y solo si para cada par de inputs $X$ y $X'$ que difieren en un solo una entrada y para todos los outputs T plausibles tenemos que:

<br>$P(M(X)\in T) = e^\epsilon P(M(X')\in T)$</br>

En nuestro caso de Randomized Response con una moneda, la condición quedaría como:

<br>$P(Y=1|X=1) = e^\epsilon P(Y=0|X=1)$</br>

<br>$\Leftrightarrow ln(\frac{P(Y=1|X=1)}{P(Y=0|X=1)}) = \epsilon$</br>





Por lo que en este caso obtenemos:

In [ ]:
epsilon = np.log(0.75 / 0.25)
print(f'Randomized Response es {epsilon}-diferencialmente privado')

Randomized Response es 1.0986122886681098-diferencialmente privado


Algunas consideraciones:

- Menores valores de epsilon implican una mayor privacidad.
- El valor de epsilon debería ser pequeño, del orden de $10^{-1}$ o $10^0$.